In [23]:
import os
import csv
import cv2
import json
import math
from time import time
from tqdm import tqdm
from itertools import product
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [24]:
import vio_ort_exp as vio_ort

In [ ]:
# Инициализация параметров
odometry = vio_ort.VIO(lat0=54.889668, lon0=83.1258973333, alt0=0)
set_dir = '2024_12_15_15_31_8_num_3'

json_files = sorted([f for f in os.listdir(set_dir) if f.endswith('.json')])
start = 1000
count_json = 100

In [26]:
# Значения для параметров
top_k_values = [512, 256]
detection_threshold_values = [0.05, 0.01]
max_iters_values = [None, 100, 300, 500]
rotation_methods = ["PIL", "CV2"]
trace_values = [8, 4]

In [27]:
# Значения для параметров
top_k_values = [512]
detection_threshold_values = [0.05]
max_iters_values = [None, 100]
rotation_methods = ["PIL"]
trace_values = [4]

In [28]:
# Генерация всех комбинаций
parameters = [
    {'top_k': top_k, 'detection_threshold': detection_threshold, 'maxIters': max_iters, 'rotation': rotation, 'trace': trace}
    for top_k, detection_threshold, max_iters, rotation, trace in product(
        top_k_values, detection_threshold_values, max_iters_values, rotation_methods, trace_values
    )
]

In [29]:
def run_vio(odometry, json_files, start, count_json, top_k, detection_threshold, maxIters, rotation, trace):
    """
    Выполняет обработку данных с использованием заданных параметров.
    """
    lat_VIO, lon_VIO, alt_VIO = [], [], []
    lat_GPS, lon_GPS, alt_GPS = [], [], []

    odometry._matcher.top_k = top_k
    odometry._matcher.detection_threshold = detection_threshold
    odometry.MAX_ITERS = maxIters
    odometry.ROTATION = rotation
    odometry.TRACE = trace

    for filename in json_files[start:start + count_json]:
        with open(f'{set_dir}/{filename}', 'r') as file:
            data = json.load(file)
            if 'GNRMC' in data and data['GNRMC']['status'] == 'A':
                img_path = os.path.join(set_dir, os.path.splitext(filename)[0] + '.jpg')
                image = cv2.imread(img_path)
                result_vio = odometry.add_trace_pt(image, data)

                lat_VIO.append(result_vio['lat'])
                lon_VIO.append(result_vio['lon'])
                alt_VIO.append(result_vio['alt'] * 1000)
                lat_GPS.append(data['GNRMC'].get('lat', 0.0))
                lon_GPS.append(data['GNRMC'].get('lon', 0.0))
                alt_GPS.append(data['GPS_RAW_INT']['alt'])

    return {
        'lat_VIO': lat_VIO,
        'lon_VIO': lon_VIO,
        'alt_VIO': alt_VIO,
        'lat_GPS': lat_GPS,
        'lon_GPS': lon_GPS,
        'alt_GPS': alt_GPS,
    }

In [30]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Функция для вычисления расстояния между двумя точками на поверхности Земли.
    """
    R = 6371000  # радиус Земли в метрах
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [31]:
def calculate_aggregated_metrics(results):
    """
    Функция для вычисления агрегированных метрик (среднее, максимальное, минимальное расстояние, RMSE).
    """
    metrics = {
        "VIO": [],
    }

    for i in range(len(results['lat_GPS'])):
        gps_lat, gps_lon = results['lat_GPS'][i], results['lon_GPS'][i]
        vio_lat, vio_lon = results['lat_VIO'][i], results['lon_VIO'][i]

        metrics["VIO"].append(haversine(gps_lat, gps_lon, vio_lat, vio_lon))

    aggregated = {
        "Metric": ["Mean Distance (m)", "Max Distance (m)", "Min Distance (m)", "RMSE (m)"],
    }

    distances = metrics["VIO"]
    mean_distance = sum(distances) / len(distances)
    max_distance = max(distances)
    min_distance = min(distances)
    rmse = math.sqrt(sum(d ** 2 for d in distances) / len(distances))

    aggregated["VIO"] = [mean_distance, max_distance, min_distance, rmse]

    return aggregated

In [32]:
def save_results_to_csv(results_all, filename):
    """
    Сохраняет результаты эксперимента в CSV файл.
    """
    with open(filename, mode='w', newline='') as f:
        writer = csv.writer(f)
        # Заголовок
        writer.writerow(["Top_k", "Detection Threshold", "Max Iters", "Rotation method", "Trace depth", "Time", "Mean Distance", "Max Distance", "RMSE"])

        # Данные
        for result in results_all:
            params = result['params']
            metrics = calculate_aggregated_metrics(result['results'])['VIO']

            writer.writerow([
                params['top_k'], params['detection_threshold'], params['maxIters'] if params['maxIters'] is not None else 'None', params['rotation'], params['trace'],
                result['time'],
                metrics[0],  # Mean Distance
                metrics[1],  # Max Distance
                metrics[3],  # RMSE
            ])

In [33]:
def transform_vio_coords(vio_lon_list, vio_lat_list):
    """
    Преобразование координат VIO в новые координаты.
    """
    vio_lon0 = vio_lon_list[0]
    vio_lat0 = vio_lat_list[0]

    vio_lon_range = max(vio_lon_list) - min(vio_lon_list)
    vio_lat_range = max(vio_lat_list) - min(vio_lat_list)

    scale_for_lon = 1 / vio_lat_range  # Широта -> долгота
    scale_for_lat = 1 / vio_lon_range  # Долгота -> широта

    transformed_lon = [(v_lat - vio_lat0) * scale_for_lon + vio_lon0 for v_lat in vio_lat_list]
    transformed_lat = [-(v_lon - vio_lon0) * scale_for_lat + vio_lat0 for v_lon in vio_lon_list]

    return transformed_lon, transformed_lat

In [34]:
def save_trajectory_plot(results, params, output_dir):
    """
    Функция для создания и сохранения графика траектории для VIO и GPS.
    """
    lat_VIO = results['lat_VIO_transformed']
    lon_VIO = results['lon_VIO_transformed']
    lat_GPS = results['lat_GPS']
    lon_GPS = results['lon_GPS']
    
    # Создаем название файла на основе параметров
    filename = f"top_k_{params['top_k']}_detection_{params['detection_threshold']}_maxIters_{params['maxIters']}_rotation_{params['rotation']}_trace_{params['trace']}.png"
    filepath = os.path.join(output_dir, filename)

    # Построение графика
    plt.figure(figsize=(10, 6))
    plt.plot(lon_GPS, lat_GPS, label='GPS', color='green', marker='o', markersize=3)
    plt.plot(lon_VIO, lat_VIO, label='VIO', color='blue', marker='x', markersize=2)
    plt.title('Trajectory Comparison (GPS vs VIO)')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend()
    plt.grid(True)

    # Сохраняем график
    plt.savefig(filepath)
    plt.close()

In [35]:
# Запуск экспериментов с разными параметрами с выводом прогресса
results_all = []
total = len(parameters)

# Создаем прогресс-бар
with tqdm(total=total, desc="Processing experiments", unit="experiment") as pbar:
    for idx, params in enumerate(parameters):
        start_time = time()
        results = run_vio(odometry, json_files, start, count_json, **params)
        elapsed_time = time() - start_time

        # Сохраняем результаты эксперимента
        results_all.append({
            'params': params,
            'results': results,
            'time': elapsed_time
        })

        # Обновляем прогресс-бар
        pbar.set_postfix({
            "Last elapsed time": f"{elapsed_time:.2f}s",
            "Completed": f"{idx + 1}/{total}"
        })
        pbar.update(1)

Processing experiments: 100%|██████████| 2/2 [02:49<00:00, 84.56s/experiment, Last elapsed time=79.61s, Completed=2/2]


In [36]:
# Применяем трансформацию и добавляем для каждого результата
output_dir = 'output_graphs'  # Папка для сохранения графиков
os.makedirs(output_dir, exist_ok=True)

for result in results_all:
    params = result['params']
    
    # Применяем трансформацию координат
    transformed_lon, transformed_lat = transform_vio_coords(
        result['results']['lon_VIO'], result['results']['lat_VIO']
    )
    
    result['results']['lon_VIO_transformed'] = transformed_lon
    result['results']['lat_VIO_transformed'] = transformed_lat

    # Сохраняем график
    save_trajectory_plot(result['results'], params, output_dir)

In [37]:
# После цикла сохраняем результаты в CSV:
save_results_to_csv(results_all, "vio_results_comparison_1000.csv")

In [38]:
def compare_trajectories(results_all):
    """
    Функция для сравнения траекторий между всеми возможными комбинациями параметров.
    """
    comparisons = []

    # Для каждой пары экспериментов
    for i, result_1 in enumerate(results_all):
        for j, result_2 in enumerate(results_all):
            if i >= j:
                continue  # Чтобы не сравнивать одинаковые эксперименты или дублировать

            params_1 = result_1['params']
            params_2 = result_2['params']

            lat_VIO_1 = result_1['results']['lat_VIO_transformed']
            lon_VIO_1 = result_1['results']['lon_VIO_transformed']

            lat_VIO_2 = result_2['results']['lat_VIO_transformed']
            lon_VIO_2 = result_2['results']['lon_VIO_transformed']

            # Вычисление расстояния между точками для двух экспериментов (VIO vs GPS)
            distances_VIO = [
                haversine(lat_VIO_1[i], lon_VIO_1[i], lat_VIO_2[i], lon_VIO_2[i])
                for i in range(len(lat_VIO_1))
            ]

            mean_distance_VIO = sum(distances_VIO) / len(distances_VIO)

            # Время выполнения для каждого эксперимента
            time_1 = result_1['time']
            time_2 = result_2['time']

            # Сравнение и сохранение результатов
            comparisons.append({
                'params_1': params_1,
                'params_2': params_2,
                'mean_distance_VIO': mean_distance_VIO,
                'time_1': time_1,
                'time_2': time_2
            })

    return comparisons

In [39]:
# Получаем все сравнения
comparisons = compare_trajectories(results_all)

In [40]:
"""# Выводим и сохраняем результаты
for comparison in comparisons:
    print(f"Comparing {comparison['params_1']} and {comparison['params_2']}:")
    print(f"Mean Distance VIO: {comparison['mean_distance_VIO']:.2f} m")
    print(f"Time 1: {comparison['time_1']:.2f} s, Time 2: {comparison['time_2']:.2f} s")
    print("-" * 30)"""

'# Выводим и сохраняем результаты\nfor comparison in comparisons:\n    print(f"Comparing {comparison[\'params_1\']} and {comparison[\'params_2\']}:")\n    print(f"Mean Distance VIO: {comparison[\'mean_distance_VIO\']:.2f} m")\n    print(f"Time 1: {comparison[\'time_1\']:.2f} s, Time 2: {comparison[\'time_2\']:.2f} s")\n    print("-" * 30)'

In [41]:
# Сохранить в CSV файл
with open('trajectory_comparisons.csv', mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Params 1', 'Params 2', 'Mean Distance VIO', 'Time 1', 'Time 2'])
    for comparison in comparisons:
        writer.writerow([
            json.dumps(comparison['params_1']),
            json.dumps(comparison['params_2']),
            comparison['mean_distance_VIO'],
            comparison['time_1'],
            comparison['time_2']
        ])

In [42]:
def create_interactive_trajectory_plot(results_all):
    fig = go.Figure()

    for result in results_all:
        params = result['params']
        lat_VIO = result['results']['lat_VIO_transformed']
        lon_VIO = result['results']['lon_VIO_transformed']
        
        # Добавляем каждую траекторию как отдельную линию на графике
        fig.add_trace(go.Scattergeo(
            lon=lon_VIO,
            lat=lat_VIO,
            mode='lines',
            name=f"Params: {params['top_k']}, {params['detection_threshold']}",
        ))
    
    lat_GPS = results_all[0]['results']['lat_GPS']
    lon_GPS = results_all[0]['results']['lon_GPS']

    fig.add_trace(go.Scattergeo(
            lon=lon_GPS,
            lat=lat_GPS,
            mode='lines',
            name="GPS",
        ))

    fig.update_layout(
        title='Trajectory Comparison (VIO)',
        geo=dict(
            projection_type='mercator',
            showland=True,
            landcolor='rgb(255, 255, 255)',
            lakecolor='rgb(255, 255, 255)',
        ),
        showlegend=True,
    )

    fig.show()


In [43]:
create_interactive_trajectory_plot(results_all)

In [44]:
lat_VIO = result['results']['lat_VIO_transformed']
lon_VIO = result['results']['lon_VIO_transformed']
print(f"{lat_VIO[0]}, {lon_VIO[0]}")
print(f"{lat_VIO[-1]}, {lon_VIO[-1]}")


54.8899846018355, 83.12534552355572
55.74549314642923, 83.88626126446604
